<a href="https://colab.research.google.com/github/mahekanna/Algo_setup/blob/main/Gann_multiverse_square_main_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####################################
# Author: Mahesh Naidu D         ####
# Version: 1.4 (Timing Fixed)    ####
# Strategy Name: Gann square 9  ####
####################################

import pandas as pd
import math
from collections import defaultdict
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.stats import skew, kurtosis
import datetime
import dateutil.rrule
import gradio as gr

def gann_square_of_9(price):
    """
    Calculates Gann Square of 9 values for a given price.
    Handles NaN values by returning an empty list.

    Args:
        price: The price to calculate Gann values for.

    Returns:
        A list of Gann Square of 9 values, or an empty list if price is NaN.
    """
    if math.isnan(price):
        return []  # Return an empty list if price is NaN

    root = math.sqrt(price)
    minus_two = root - 2
    rounded = math.ceil(minus_two)

    gann_values = []
    for i in range(24):
        x = rounded + 0.125 * (i + 1)
        gann_values.append(round(x * x, 2))  # Round to 2 decimal places

    return gann_values

def find_buy_sell_levels(price, gann_values):
    """
    Finds the buy above and sell below levels from Gann values.

    Args:
        price: The current price.
        gann_values: A list of Gann Square of 9 values.

    Returns:
        A tuple containing the buy above and sell below levels.
    """
    min_positive_index = -1
    for i, value in enumerate(gann_values):
        if value >= price:
            min_positive_index = i
            break

    if min_positive_index >= 0:
        buy_above = gann_values[min_positive_index]
        sell_below = gann_values[min_positive_index - 1]
    else:
        buy_above = None
        sell_below = None

    return buy_above, sell_below

def calculate_targets(entry_level, gann_values, direction='long'):
    """
    Calculates target levels based on trade direction.

    Args:
        entry_level: The entry level (buy_above or sell_below).
        gann_values: A list of Gann Square of 9 values.
        direction: 'long' or 'short' indicating the trade direction.

    Returns:
        A list of target levels.
    """
    targets = []
    try:
        start_index = gann_values.index(entry_level)
        if direction == 'long':
            targets = gann_values[start_index + 1: start_index + 5]
        elif direction == 'short':
            targets = gann_values[start_index - 1: start_index - 5: -1]  # Reverse for short
    except ValueError:
        pass
    return targets

def gann_strategy(price_data, timeframe):
    """
    Implements the Gann strategy with target levels and exit conditions.

    Args:
        price_data: A pandas DataFrame with 'close' price data and a datetime index.
        timeframe: The timeframe of the price data (e.g., '5min', '15min', '30min', '1H').

    Returns:
        A list of signals ('buy', 'sell', 'exit') and target levels.
    """
    signals = []
    for i in range(1, len(price_data) - 1):
        current_price = price_data['close'][i]
        previous_price = price_data['close'][i - 1]

        gann_values = gann_square_of_9(previous_price)
        buy_above, sell_below = find_buy_sell_levels(previous_price, gann_values)

        if buy_above and current_price >= buy_above:
            buy_targets = calculate_targets(buy_above, gann_values, direction='long')
            # Use current candle close time
            signals.append(('buy', buy_above, buy_targets, price_data.index[i]))
        elif sell_below and current_price <= sell_below:
            sell_targets = calculate_targets(sell_below, gann_values, direction='short')
            # Use current candle close time
            signals.append(('sell', sell_below, sell_targets, price_data.index[i]))

        # ... (your exit conditions - same as before)
        # Basic exit conditions
        if signals and signals[-1][0] == 'buy':
            for j, target in enumerate(signals[-1][2]):
                if current_price >= target:
                    signals.append(('exit_buy', target, j+1, price_data.index[i]))
        elif signals and signals[-1][0] == 'sell':
            for j, target in enumerate(signals[-1][2]):
                if current_price <= target:
                    signals.append(('exit_sell', target, j+1, price_data.index[i]))


    return signals

def calculate_quantity(capital, entry_price, risk_percentage, stop_loss_price):
    """
    Calculates the quantity of shares/lots to trade based on risk management.

    Args:
        capital: The total trading capital.
        entry_price: The entry price of the trade.
        risk_percentage: The percentage of capital to risk on the trade.
        stop_loss_price: The stop-loss price for the trade.

    Returns:
        The calculated quantity (integer).
    """
    risk_amount = capital * (risk_percentage / 100)
    risk_per_share = abs(entry_price - stop_loss_price)
    quantity = math.floor(risk_amount / risk_per_share)  # Round down
    return quantity

#Metric defination
def calculate_metrics(signals, price_data):
    """
    Calculates trading metrics based on the generated signals.

    Args:
        signals: A list of signals generated by the gann_strategy function.
        price_data: A pandas DataFrame with 'close' price data and a datetime index.

    Returns:
        A dictionary of trading metrics.
    """
    metrics = defaultdict(int)
    long_trades = []
    short_trades = []
    long_pnl = []
    short_pnl = []
    all_pnls = []  # List to store all PnLs

    position = 0  # 1 for long, -1 for short, 0 for no position
    entry_price = 0
    entry_time = None
    entry_quantity = 0  # Store the entry quantity

    for signal, entry_level, targets, timestamp, quantity in signals:  # Include quantity
        try:  # Add a try-except block to handle potential errors
            if signal == 'buy' and position == 0:
                position = 1
                entry_price = entry_level
                entry_time = timestamp
                entry_quantity = quantity  # Store the quantity
                long_trades.append({'entry_time': entry_time, 'entry_price': entry_price, 'targets': targets, 'quantity': quantity})
            elif signal == 'sell' and position == 0:
                position = -1
                entry_price = entry_level
                entry_time = timestamp
                entry_quantity = quantity  # Store the quantity
                short_trades.append({'entry_time': entry_time, 'entry_price': entry_price, 'targets': targets, 'quantity': quantity})
            elif signal.startswith('exit') and position != 0:
                exit_price = price_data['close'].loc[timestamp]  # Get close price at exit time
                pnl = (exit_price - entry_price) * entry_quantity if position == 1 else (entry_price - exit_price) * entry_quantity  # Calculate PnL with quantity
                all_pnls.append(pnl)  # Add PnL to the list

                if position == 1:
                    long_trades[-1]['exit_time'] = timestamp
                    long_trades[-1]['exit_price'] = exit_price
                    long_trades[-1]['pnl'] = pnl
                    metrics['long_trades'] += 1
                    metrics['long_pnl'] += pnl
                    long_pnl.append(pnl)
                else:
                    short_trades[-1]['exit_time'] = timestamp
                    short_trades[-1]['exit_price'] = exit_price
                    short_trades[-1]['pnl'] = pnl
                    metrics['short_trades'] += 1
                    metrics['short_pnl'] += pnl
                    short_pnl.append(pnl)

                position = 0
        except Exception as e:
            print(f"Error processing signal: {signal}, Error: {e}")

    # Calculate other metrics
    metrics['total_trades'] = metrics['long_trades'] + metrics['short_trades']
    metrics['total_pnl'] = metrics['long_pnl'] + metrics['short_pnl']

    if metrics['long_trades'] > 0:
        metrics['avg_long_pnl'] = metrics['long_pnl'] / metrics['long_trades']
        long_wins = sum(1 for p in long_pnl if p > 0)
        metrics['long_win_rate'] = long_wins / metrics['long_trades'] if metrics['long_trades'] > 0 else 0
    else:
        metrics['avg_long_pnl'] = 0
        metrics['long_win_rate'] = 0

    if metrics['short_trades'] > 0:
        metrics['avg_short_pnl'] = metrics['short_pnl'] / metrics['short_trades']
        short_wins = sum(1 for p in short_pnl if p > 0)
        metrics['short_win_rate'] = short_wins / metrics['short_trades'] if metrics['short_trades'] > 0 else 0
    else:
        metrics['avg_short_pnl'] = 0
        metrics['short_win_rate'] = 0

    metrics['winning_trades'] = long_wins + short_wins
    metrics['losing_trades'] = metrics['total_trades'] - metrics['winning_trades']
    metrics['winning_percentage'] = metrics['winning_trades'] / metrics['total_trades'] if metrics['total_trades'] > 0 else 0

    # Calculate Sharpe and Sortino ratios (assuming a risk-free rate of 0)
    if len(all_pnls) > 1:
        returns = pd.Series(all_pnls)
        metrics['sharpe_ratio'] = returns.mean() / returns.std() if returns.std() != 0 else 0
        negative_returns = returns[returns < 0]
        metrics['sortino_ratio'] = returns.mean() / negative_returns.std() if negative_returns.std() != 0 else 0
    else:
        metrics['sharpe_ratio'] = 0
        metrics['sortino_ratio'] = 0

    return metrics, long_trades, short_trades, all_pnls


#Plotting candles function
#def plot_candlestick_chart(df, signals, timeframe, scrip_symbol, save_path):
def plot_candlestick_chart(df, signals, timeframe, scrip_symbol, save_path):
    """
    Plots a candlestick chart with buy/sell signals and saves it to a file.

    Args:
        df: The DataFrame containing OHLCV data.
        signals: The list of signals generated by the strategy.
        timeframe: The timeframe of the data.
        scrip_symbol: The symbol of the scrip (stock).
        save_path: The path to save the chart image.
    """
    fig, ax = plt.subplots(figsize=(12, 6))

    # Plot candlesticks
    width = 0.6
    width2 = 0.1
    up = df[df.close >= df.open]
    down = df[df.close < df.open]
    ax.bar(up.index, up.close - up.open, width, bottom=up.open, color="green")
    ax.bar(up.index, up.high - up.close, width2, bottom=up.close, color="green")
    ax.bar(up.index, up.low - up.open, width2, bottom=up.open, color="green")
    ax.bar(down.index, down.close - down.open, width, bottom=down.open, color="red")
    ax.bar(down.index, down.high - down.open, width2, bottom=down.open, color="red")
    ax.bar(down.index, down.low - down.close, width2, bottom=down.close, color="red")

    # Plot buy/sell signals
    for signal, entry_level, targets, timestamp, quantity in signals:
        if signal == 'buy':
            ax.plot(timestamp, entry_level, '^', markersize=10, color='g', label='Buy')
        elif signal == 'sell':
            ax.plot(timestamp, entry_level, 'v', markersize=10, color='r', label='Sell')

    ax.set_title(f"Candlestick Chart with Gann Signals ({timeframe})")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price")
    ax.legend()

    # Save the chart
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    plt.savefig(os.path.join(save_path, f"{scrip_symbol}_{timeframe}.png"))
    plt.close(fig)  # Close the figure to free up memory


#def plot_pnl_curve(pnls, timeframe, scrip_symbol, save_path):
def plot_pnl_curve(pnls, timeframe, scrip_symbol, save_path):
    """
    Plots the PnL curve and saves it to a file.

    Args:
        pnls: A list of PnLs.
        timeframe: The timeframe of the data.
        scrip_symbol: The symbol of the scrip (stock).
        save_path: The path to save the PnL curve image.
    """
    cumulative_pnls = np.cumsum(pnls)  # Calculate cumulative PnLs for this timeframe
    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_pnls)
    plt.title(f"PnL Curve ({timeframe})")
    plt.xlabel("Trades")
    plt.ylabel("Cumulative PnL")

    # Save the PnL curve
    plt.savefig(os.path.join(save_path, f"{scrip_symbol}_{timeframe}_pnl_curve.png"))
    plt.close()


def resample_data(df, timeframe):
    """
    Resamples the DataFrame to the specified timeframe with custom intervals.
    """
    if timeframe == '30min':
        # Custom 30-minute intervals
        df['time_group'] = df.index.floor('30min') + pd.Timedelta(minutes=15)
        df = df.groupby('time_group').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
    elif timeframe == '1H':
        # Custom 1-hour intervals
        df['time_group'] = df.index.floor('1H') + pd.Timedelta(minutes=15)
        df = df.groupby('time_group').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
    else:
        # Standard resampling
        df = df.resample(timeframe).agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'volume': 'sum'})
    return df

def analyze_stock(stock_name, timeframe='5min'):
    """
    Analyzes the selected stock and timeframe using the Gann strategy.

    Args:
        stock_name: The name of the stock (folder name).
        timeframe: The selected timeframe.

    Returns:
        A tuple containing the metrics DataFrame, trades DataFrame,
        candlestick chart image path, and PNL curve image path.
    """
    # ... (file path setup - same as before)
    data_dir = "/content/drive/MyDrive/BACKTEST/stock-1min/stock_min_data"
    stock_path = os.path.join(data_dir, stock_name)

    # Construct file paths
    file_path = os.path.join(stock_path, f"{stock_name}.parquet")
    save_path = os.path.join(data_dir, f"Gann_Charts/{stock_name}")
    metrics_path = os.path.join(save_path, f"{stock_name}_metrics.csv")
    trades_path = os.path.join(save_path, f"{stock_name}_{timeframe}_trades.csv")
    chart_path = os.path.join(save_path, f"{stock_name}_{timeframe}.png")
    pnl_path = os.path.join(save_path, f"{stock_name}_{timeframe}_pnl_curve.png")


    # Read and preprocess data
    df = pd.read_parquet(file_path)
    df = df.rename(columns={'datetime': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    # Resample data
    resampled_df = resample_data(df, timeframe)

    # Apply Gann strategy
    signals = gann_strategy(resampled_df, timeframe)

    # ... (rest of the analyze_stock function - same as before)
    # Modify signals to include quantity
    updated_signals = []
    for signal, entry_level, targets, timestamp in signals:
        if signal == 'buy':
            stop_loss_price = entry_level * 0.98  # Example stop-loss
            quantity = calculate_quantity(capital, entry_level, risk_percentage, stop_loss_price)
            updated_signals.append(('buy', entry_level, targets, timestamp, quantity))
        elif signal == 'sell':
            stop_loss_price = entry_level * 1.02  # Example stop-loss
            quantity = calculate_quantity(capital, entry_level, risk_percentage, stop_loss_price)
            updated_signals.append(('sell', entry_level, targets, timestamp, quantity))
        else:
            updated_signals.append((signal, entry_level, targets, timestamp, None))

    signals = updated_signals

    # Calculate metrics
    metrics, long_trades, short_trades, all_pnls = calculate_metrics(signals, resampled_df)

    # Store metrics and trades
    all_metrics = {timeframe: metrics}
    all_trades = {timeframe: {'long_trades': long_trades, 'short_trades': short_trades}}

    # Calculate cumulative profit
    cumulative_profit = 0
    for trade in long_trades + short_trades:
        cumulative_profit += trade.get('pnl', 0)  # Handle potential KeyError
        trade['cumulative_profit'] = cumulative_profit

    # Plot charts
    plot_candlestick_chart(resampled_df, signals, timeframe, stock_name, save_path)
    plot_pnl_curve(all_pnls, timeframe, stock_name, save_path)

    # Save results
    metrics_df = pd.DataFrame(all_metrics).transpose()
    metrics_df.to_csv(metrics_path, index=False)

    long_trades_df = pd.DataFrame(all_trades[timeframe]['long_trades'])
    short_trades_df = pd.DataFrame(all_trades[timeframe]['short_trades'])
    long_trades_df['trade_type'] = 'long'
    short_trades_df['trade_type'] = 'short'
    all_trades_df = pd.concat([long_trades_df, short_trades_df], ignore_index=True)
    all_trades_df.to_csv(trades_path, index=False)

    # Load and return results
    metrics_df = pd.read_csv(metrics_path)
    trades_df = pd.read_csv(trades_path)
    return metrics_df, trades_df, chart_path, pnl_path


# --- Main Execution ---
# Get the list of stocks (folder names)
stock_list = [f for f in os.listdir("/content/drive/MyDrive/BACKTEST/stock-1min/stock_min_data") if os.path.isdir(os.path.join("/content/drive/MyDrive/BACKTEST/stock-1min/stock_min_data", f))]

# Set your initial capital and risk percentage
capital = 100000  # Set your initial capital
risk_percentage = 1  # Set the percentage of capital to risk per trade

# Create the Gradio interface
iface = gr.Interface(
    fn=analyze_stock,
    inputs=[
        gr.Dropdown(choices=stock_list, label="Select Stock"),
        gr.Dropdown(choices=['5min', '15min', '30min', '1H'], label="Select Timeframe", value='5min')
    ],
    outputs=[
        gr.Dataframe(label="Metrics"),
        gr.Dataframe(label="Trades"),
        gr.Image(label="Candlestick Chart", type="filepath"),
        gr.Image(label="PNL Curve", type="filepath")
    ],
    title="Gann Strategy Backtesting"
)

iface.launch(debug=True)

# --- Main Execution ---
# ... (Gradio interface setup - same as before)

In [4]:
import pandas as pd
import altair as alt

def analyze_trades(file_path):
    """
    Analyzes trade data to generate daily, weekly, and monthly PNL,
    cumulative PNL charts, and daily profit/loss visualization.

    Args:
      file_path: Path to the CSV file containing trade data.
                 The file should have columns named 'entry_time' and 'pnl'.
    """
    # Load the dataframe
    df = pd.read_csv('/content/drive/MyDrive/BACKTEST/stock-1min/stock_min_data/Gann_Charts/AMARAJ/AMARAJ_1H_trades.csv')

    # Convert `entry_time` to datetime object
    df['entry_time'] = pd.to_datetime(df['entry_time'])

    # Resample the data to daily, weekly, and monthly frequencies
    daily_pnl = df.set_index('entry_time')['pnl'].resample('D').sum()
    weekly_pnl = df.set_index('entry_time')['pnl'].resample('W').sum()
    monthly_pnl = df.set_index('entry_time')['pnl'].resample('M').sum()

    # Calculate cumulative PNL for daily, weekly, and monthly data
    daily_cumulative_pnl = daily_pnl.cumsum()
    weekly_cumulative_pnl = weekly_pnl.cumsum()
    monthly_cumulative_pnl = monthly_pnl.cumsum()

    # Plot the cumulative PNL for daily, weekly, and monthly data
    daily_chart = alt.Chart(daily_cumulative_pnl.reset_index()).mark_line().encode(
        x=alt.X('entry_time', axis=alt.Axis(format='%Y-%m-%d'), title='Date'),
        y=alt.Y('pnl', title='Cumulative PnL'),
        tooltip=[alt.Tooltip('entry_time', format='%Y-%m-%d'), 'pnl']
    ).properties(
        title='Daily Cumulative PnL'
    ).interactive()

    weekly_chart = alt.Chart(weekly_cumulative_pnl.reset_index()).mark_line().encode(
        x=alt.X('entry_time', axis=alt.Axis(format='%Y-%m-%d'), title='Week'),
        y=alt.Y('pnl', title='Cumulative PnL'),
        tooltip=[alt.Tooltip('entry_time', format='%Y-%m-%d'), 'pnl']
    ).properties(
        title='Weekly Cumulative PnL'
    ).interactive()

    monthly_chart = alt.Chart(monthly_cumulative_pnl.reset_index()).mark_line().encode(
        x=alt.X('entry_time', axis=alt.Axis(format='%Y-%m-%d'), title='Month'),
        y=alt.Y('pnl', title='Cumulative PnL'),
        tooltip=[alt.Tooltip('entry_time', format='%Y-%m-%d'), 'pnl']
    ).properties(
        title='Monthly Cumulative PnL'
    ).interactive()

    # Combine the plots
    daily_chart.save('daily_cumulative_pnl.json')
    weekly_chart.save('weekly_cumulative_pnl.json')
    monthly_chart.save('monthly_cumulative_pnl.json')

    # Calculate daily profit and loss
    daily_profit = df[df['pnl'] > 0].set_index('entry_time')['pnl'].resample('D').sum()
    daily_loss = df[df['pnl'] < 0].set_index('entry_time')['pnl'].resample('D').sum()

    # Concatenate the `daily_profit` and `daily_loss` Series into a DataFrame
    df_profit_loss = pd.concat([
        daily_profit.to_frame().assign(Category='Profit'),
        daily_loss.to_frame().assign(Category='Loss')
    ]).reset_index()

    # Create a bar chart with `entry_time` on the x-axis and `pnl` on the y-axis
    chart = alt.Chart(df_profit_loss).mark_bar().encode(
        x=alt.X('entry_time', title='Date'),
        y=alt.Y('pnl', title='Profit and Loss'),
        color=alt.Color('Category', scale={'domain': ['Profit', 'Loss'], 'range': ['green', 'red']}),
        tooltip=['entry_time', 'pnl', 'Category']
    ).properties(
        title='Daily Profit and Loss'
    ).interactive()

    # Save the chart
    chart.save('daily_profit_loss.json')

In [6]:
import pandas as pd
import altair as alt

def analyze_trades(df):
    """
    Analyzes trade data to generate daily, weekly, and monthly PNL,
    cumulative PNL charts, and daily profit/loss visualization.

    Args:
      df: DataFrame containing trade data with 'entry_time' and 'pnl' columns.

    Returns:
      A tuple containing the daily, weekly, and monthly cumulative PNL charts
      and the daily profit/loss chart.
    """
    # Convert `entry_time` to datetime object
    df['entry_time'] = pd.to_datetime(df['entry_time'])

    # Resample the data to daily, weekly, and monthly frequencies
    daily_pnl = df.set_index('entry_time')['pnl'].resample('D').sum()
    weekly_pnl = df.set_index('entry_time')['pnl'].resample('W').sum()
    monthly_pnl = df.set_index('entry_time')['pnl'].resample('M').sum()

    # Calculate cumulative PNL for daily, weekly, and monthly data
    daily_cumulative_pnl = daily_pnl.cumsum()
    weekly_cumulative_pnl = weekly_pnl.cumsum()
    monthly_cumulative_pnl = monthly_pnl.cumsum()

    # Plot the cumulative PNL for daily, weekly, and monthly data
    daily_chart = alt.Chart(daily_cumulative_pnl.reset_index()).mark_line().encode(
        x=alt.X('entry_time', axis=alt.Axis(format='%Y-%m-%d'), title='Date'),
        y=alt.Y('pnl', title='Cumulative PnL'),
        tooltip=[alt.Tooltip('entry_time', format='%Y-%m-%d'), 'pnl']
    ).properties(
        title='Daily Cumulative PnL'
    ).interactive()

    weekly_chart = alt.Chart(weekly_cumulative_pnl.reset_index()).mark_line().encode(
        x=alt.X('entry_time', axis=alt.Axis(format='%Y-%m-%d'), title='Week'),
        y=alt.Y('pnl', title='Cumulative PnL'),
        tooltip=[alt.Tooltip('entry_time', format='%Y-%m-%d'), 'pnl']
    ).properties(
        title='Weekly Cumulative PnL'
    ).interactive()

    monthly_chart = alt.Chart(monthly_cumulative_pnl.reset_index()).mark_line().encode(
        x=alt.X('entry_time', axis=alt.Axis(format='%Y-%m-%d'), title='Month'),
        y=alt.Y('pnl', title='Cumulative PnL'),
        tooltip=[alt.Tooltip('entry_time', format='%Y-%m-%d'), 'pnl']
    ).properties(
        title='Monthly Cumulative PnL'
    ).interactive()

    # Calculate daily profit and loss
    daily_profit = df[df['pnl'] > 0].set_index('entry_time')['pnl'].resample('D').sum()
    daily_loss = df[df['pnl'] < 0].set_index('entry_time')['pnl'].resample('D').sum()

    # Concatenate the `daily_profit` and `daily_loss` Series into a DataFrame
    df_profit_loss = pd.concat([
        daily_profit.to_frame().assign(Category='Profit'),
        daily_loss.to_frame().assign(Category='Loss')
    ]).reset_index()

    # Create a bar chart with `entry_time` on the x-axis and `pnl` on the y-axis
    profit_loss_chart = alt.Chart(df_profit_loss).mark_bar().encode(
        x=alt.X('entry_time', title='Date'),
        y=alt.Y('pnl', title='Profit and Loss'),
        color=alt.Color('Category', scale={'domain': ['Profit', 'Loss'], 'range': ['green', 'red']}),
        tooltip=['entry_time', 'pnl', 'Category']
    ).properties(
        title='Daily Profit and Loss'
    ).interactive()

    return daily_chart, weekly_chart, monthly_chart, profit_loss_chart

# Load the dataframe
df = pd.read_csv('/content/drive/MyDrive/BACKTEST/stock-1min/stock_min_data/Gann_Charts/AMARAJ/AMARAJ_1H_trades.csv')

# Get the charts
daily_chart, weekly_chart, monthly_chart, profit_loss_chart = analyze_trades(df)

# Display the charts
daily_chart.display()
weekly_chart.display()
monthly_chart.display()
profit_loss_chart.display()

<ipython-input-6-19448cb473ce>:22: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_pnl = df.set_index('entry_time')['pnl'].resample('M').sum()


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

/usr/local/lib/python3.10/dist-packages/altair/utils/core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)